In [1]:
import os
import sqlite3
from flask import Flask, render_template, send_from_directory

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'pdfs'
app.config['DATABASE'] = 'pdfs.db'

# Define the database schema and create a connection
def create_db():
    conn = sqlite3.connect(app.config['DATABASE'])
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS pdf_files
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  filename TEXT NOT NULL,
                  path TEXT NOT NULL)''')
    conn.commit()
    conn.close()

# Add PDF files to the database
def add_pdf_files_to_db():
    conn = sqlite3.connect(app.config['DATABASE'])
    c = conn.cursor()
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        if filename.endswith('.pdf'):
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            secure_filename = filename.replace(' ', '_')
            c.execute("INSERT INTO pdf_files (filename, path) VALUES (?, ?)",
                      (secure_filename, filepath))
    conn.commit()
    conn.close()

# Retrieve PDF files from the database
def get_pdf_files_from_db():
    conn = sqlite3.connect(app.config['DATABASE'])
    c = conn.cursor()
    c.execute("SELECT filename, path FROM pdf_files")
    pdf_files = c.fetchall()
    conn.close()
    return pdf_files

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/pdfs')
def pdfs():
    pdf_files = get_pdf_files_from_db()
    return render_template('pdfs.html', pdf_files=pdf_files)

@app.route('/pdfs/<filename>')
def serve_pdf(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
    create_db()
    add_pdf_files_to_db()
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Apr/2023 20:33:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:33:16] "GET /pdfs HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:33:17] "GET /pdfs/Chapter_0_-__Computer_Architecture.pdf HTTP/1.1" 404 -
127.0.0.1 - - [23/Apr/2023 20:33:36] "GET /pdfs/Chapter_9_-__File_Processing.pdf HTTP/1.1" 404 -


In [2]:
import os
from flask import Flask, render_template, send_from_directory, url_for, g
from werkzeug.utils import secure_filename
import sqlite3

app = Flask(__name__)

# Define the directory where PDF files are stored
PDF_DIR = 'pdfs'

# Define the name of the SQLite database file
DB_FILENAME = 'pdfs.db'

# Define the full path to the database file
DB_PATH = os.path.join(app.root_path, DB_FILENAME)

# Define the database schema
SCHEMA = '''
CREATE TABLE IF NOT EXISTS pdf_files (
    id INTEGER PRIMARY KEY,
    filename TEXT UNIQUE,
    filepath TEXT
);
'''

# Create a connection to the database
conn = sqlite3.connect(DB_PATH)

# Create the pdf_files table if it doesn't already exist
with conn:
    conn.execute(SCHEMA)

# Add all PDF files to the database
for filename in os.listdir(PDF_DIR):
    if filename.endswith('.pdf'):
        filepath = os.path.join(PDF_DIR, filename)
        securefilename = secure_filename(filename)
        with conn:
            conn.execute('INSERT OR IGNORE INTO pdf_files (filename, filepath) VALUES (?, ?);', (securefilename, filepath))

@app.teardown_appcontext
def close_db(error):
    if 'db' in g:
        g.db.close()

def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect('pdfs.db')
    return g.db

# Define the route for serving PDF files
@app.route('/pdfs/<path:filename>')
def serve_pdf(filename):
    return send_from_directory(PDF_DIR, filename)

@app.route('/')
def index():
    return render_template('index.html')

# Define the route for the PDF list page

@app.route('/pdfs')
def pdfs():
    conn = get_db()
    pdf_files = conn.execute('SELECT filename, filepath FROM pdf_files').fetchall()
    return render_template('pdfs.html', pdf_files=pdf_files)

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/contact')
def contact():
    return render_template('contact.html')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Apr/2023 20:53:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:53:37] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:53:38] "GET /contact HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:53:39] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:53:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:53:41] "GET /pdfs HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:53:48] "GET /pdfs/Chapter_6_-_Arrays.pdf HTTP/1.1" 200 -


In [2]:
import os
from werkzeug.utils import secure_filename

PDF_DIR = 'pdfs'

for filename in os.listdir(PDF_DIR):
    if filename.endswith('.pdf'):
        old_path = os.path.join(PDF_DIR, filename)
        new_filename = secure_filename(filename)
        new_path = os.path.join(PDF_DIR, new_filename)
        os.rename(old_path, new_path)
        print(f'Renamed {filename} to {new_filename}')


Renamed Chapter 0 -  Computer Architecture.pdf to Chapter_0_-_Computer_Architecture.pdf
Renamed Chapter 1 -  Introduction.pdf to Chapter_1_-_Introduction.pdf
Renamed Chapter 10 -  Data Capturing and Correctness.pdf to Chapter_10_-_Data_Capturing_and_Correctness.pdf
Renamed Chapter 11 -  Testing.pdf to Chapter_11_-_Testing.pdf
Renamed Chapter 12 -  Program Testing.pdf to Chapter_12_-_Program_Testing.pdf
Renamed Chapter 13 -  Algorithm Complexity.pdf to Chapter_13_-_Algorithm_Complexity.pdf
Renamed Chapter 14A -  Introduction to sorting Algorithms.pdf to Chapter_14A_-_Introduction_to_sorting_Algorithms.pdf
Renamed Chapter 14B -  Quicksort.pdf to Chapter_14B_-_Quicksort.pdf
Renamed Chapter 15 -  Introduction to Searching algorithms.pdf to Chapter_15_-_Introduction_to_Searching_algorithms.pdf
Renamed Chapter 16 -  Hashing.pdf to Chapter_16_-_Hashing.pdf
Renamed Chapter 2 -  Program Flowchart.pdf to Chapter_2_-_Program_Flowchart.pdf
Renamed Chapter 3 -  Dry Running of Algorithms.pdf to Chap